In [1]:
# Specify hyperparameters
num_epochs = 50
learning_rate = 0.0005
momentum = 0.9
batch_size = 25
num_workers = 0

# Specify model to train
model_name = "InceptionV3"
assert model_name in ["ResNet18", "ResNet50", "DenseNet121", "SqueezeNet1.1", "InceptionV3"]

In [2]:
import torch.nn as nn
import torchvision

def no_grad(model):
    for param in model.parameters():
        param.requires_grad = False
        
    return model

if model_name == "ResNet18":
    trained_model_path = 'models/resnet18_pretrained.pt'
    model_conv = no_grad(torchvision.models.resnet18(pretrained=True))
    
    num_ftrs = model_conv.fc.in_features
    model_conv.fc = nn.Linear(num_ftrs, 2)
    
elif model_name == "ResNet50":
    trained_model_path = 'models/resnet50_pretrained.pt'
    model_conv = no_grad(torchvision.models.resnet50(pretrained=True))
    
    num_ftrs = model_conv.fc.in_features
    model_conv.fc = nn.Linear(num_ftrs, 2)
         
elif model_name == "DenseNet121":
    trained_model_path = 'models/densenet121_pretrained.pt'
    model_conv = no_grad(torchvision.models.densenet121(pretrained=True))
    
    num_ftrs = model_conv.classifier.in_features
    model_conv.classifier = nn.Linear(num_ftrs, 2)
    
elif model_name == "SqueezeNet1.1":
    trained_model_path = 'models/squeezenet1-1_pretrained.pt'
    model_conv = no_grad(torchvision.models.squeezenet1_1(pretrained=True))
    
    # SqueezeNet fine tuning adapted from https://discuss.pytorch.org/t/fine-tuning-squeezenet/3855
    num_ftrs = 512
    model_conv.classifier = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Conv2d(num_ftrs, 2, kernel_size=1),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(13)
    )
    
elif model_name == "InceptionV3":
    trained_model_path = 'models/inceptionv3_pretrained.pt'
    model_conv = no_grad(torchvision.models.inception_v3(pretrained=True))
    
    num_ftrs = model_conv.AuxLogits.fc.in_features
    model_conv.AuxLogits.fc = nn.Linear(num_ftrs, 2)  
    num_ftrs = model_conv.fc.in_features        
    model_conv.fc = nn.Linear(num_ftrs, 2)



In [3]:
# Data transformations and subsequent training adapted from https://github.com/shervinmin/DeepCovid
from torchvision import transforms

img_size = 299 if (model_name == "InceptionV3") else 224

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(1024),
        transforms.CenterCrop(512),
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(1024),
        transforms.CenterCrop(512),
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [4]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

data_dir = './data/'

image_datasets = {x: ImageFolder(os.path.join(data_dir, x),
                                 data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
                             shuffle=True, num_workers=num_workers)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes  ## 0: child, and 1: nonchild

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
import copy
import time
import torch
from matplotlib import pyplot as plt
from termcolor import colored
from IPython.display import clear_output


def train_model(name, model, criterion, optimizer, scheduler):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc  = 0.0
    train_epoch_loss = []
    valid_epoch_loss = []

    for epoch in range(num_epochs):
        clear_output(wait=True)
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10, "\n")

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_prec= 0.0
            running_rec = 0.0
            running_f1  = 0.0

            # Iterate over data.
            for cur_batch_ind, (inputs, labels) in enumerate(dataloaders[phase], start=1):
                batch_start_time = time.time()

                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                if not isinstance(model, torchvision.models.inception.Inception3):
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                else:
                    with torch.set_grad_enabled(phase == 'train'):
                        if phase == 'train':
                            outputs, aux_outputs = model(inputs)

                            loss1 = criterion(outputs, labels)
                            loss2 = criterion(aux_outputs, labels)
                            loss = loss1 + 0.4*loss2
                        else:
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                matches_bool = (preds == labels.data).cpu()
                matches_sum = matches_bool.sum()
                running_corrects += matches_bool.sum()

                cur_acc = matches_sum.double() / batch_size
                batch_time = time.time() - batch_start_time
                comparison = "".join(colored("=", "green") if b else colored("X", "red") for b in matches_bool)
                print("%d-th epoch, %d-th batch (size=%d), %s acc= %.3f, %.3f s: %s" %(epoch+1, cur_batch_ind, len(labels), phase, cur_acc, batch_time, comparison ))

            epoch_loss= running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}\n'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())

            if phase == 'train':
                train_epoch_loss.append(epoch_loss)
            else:
                valid_epoch_loss.append(epoch_loss)

        plt.clf()
        plt.plot(range(len(train_epoch_loss)), train_epoch_loss, label="train loss")
        plt.plot(range(len(valid_epoch_loss)), valid_epoch_loss, label="valid loss")
        plt.suptitle(name)
        plt.xlabel("epoch")
        plt.ylabel("errors")
        plt.legend()
        plt.savefig(f"{name}_loss.png")
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc= %.3f at Epoch: %d' %(best_acc,best_epoch) )

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
import time
from torch import optim
from torch.optim import lr_scheduler
from torchvision import models

start_time = time.time()

model_conv = model_conv.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as opposed to before
if (model_name == "ResNet18") or (model_name == "ResNet50"):
    optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=learning_rate, momentum=momentum)
elif (model_name == "DenseNet121") or (model_name == "SqueezeNet1.1"):
    optimizer_conv = optim.SGD(model_conv.classifier.parameters(), lr=learning_rate, momentum=momentum)
elif model_name == "InceptionV3":
    params_to_update = [p for _, p in model_conv.named_parameters() if p.requires_grad]
    optimizer_conv = optim.SGD(params_to_update, lr=learning_rate, momentum=momentum)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_name, model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler)
model_conv.eval()
torch.save(model_conv, trained_model_path)

end_time= time.time()
print("total_time tranfer learning=", end_time - start_time)
